In [1]:
# import the package
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import requests
import json
import time
from pprint import pprint
# Google developer API key
from config import api_key
from folium.plugins import HeatMap
import branca.colormap as cm
from folium import plugins
from glob import glob
import plotly.graph_objects as go
import branca.colormap as cmp

In [2]:
# import the data 
# load in csv
Fyi_Salary = pd.read_csv("./data/Levels_Fyi_Salary_Data.csv")
Fyi_Salary.info()
Fyi_Salary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  float64
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [3]:
# create the new dataset
new_Fyi_Salary = Fyi_Salary[['company','title','totalyearlycompensation','location','yearsatcompany','basesalary',]]
new_Fyi_Salary = new_Fyi_Salary.dropna(how = 'any')
new_Fyi_Salary.info()  
cities = new_Fyi_Salary["location"].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62637 entries, 0 to 62641
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  62637 non-null  object 
 1   title                    62637 non-null  object 
 2   totalyearlycompensation  62637 non-null  int64  
 3   location                 62637 non-null  object 
 4   yearsatcompany           62637 non-null  float64
 5   basesalary               62637 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.3+ MB


In [4]:
new_Fyi_Salary_citygroup = new_Fyi_Salary.groupby('location')
count = new_Fyi_Salary_citygroup["company"].count()
average_comp = new_Fyi_Salary_citygroup["totalyearlycompensation"].mean()


### APIs (Lat and Lng)

In [5]:
# URL
# lat = []
# lng = []

# for city in cities:
#     start = time.time()
#     # Target city
#     target_city = city
#     # Build the endpoint URL
#     target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={api_key}"
#     # Run a request to endpoint and convert result to json
#     geo_data = requests.get(target_url).json()
#     lat.append(geo_data['results'][0]['geometry']['location']['lat'])
#     lng.append(geo_data['results'][0]['geometry']['location']['lng'])
#     # Print the latitude and longitude
#     # print(lat, lng)
#     # print(city)
#     #print(len(lat))

In [5]:
# marker_df = pd.DataFrame({
#     "location" : cities,
#     "latitude" : lat,
#     "longitude" : lng,  
# }) 
# marker_df
# marker_df.to_csv("marker_df1.csv", index = False, header = True)

marker_df = pd.read_csv("./data/marker_df1.csv")
marker_df.head()

,location,latitude,longitude
0,"Redwood City, CA",37.484795,-122.228141
1,"San Francisco, CA",37.774929,-122.419415
2,"Seattle, WA",47.606209,-122.332071
3,"Sunnyvale, CA",37.368830,-122.036350
4,"Mountain View, CA",37.386052,-122.083851


### Q4: What areas (cities) have the most demand for data scientists?


In [6]:
# merge the data 
merge_df = pd.merge(marker_df,count, on = "location")
merge_df = merge_df.rename(columns = {'company':'count'})  

group_count = []
for item in merge_df["count"]:  
# group the count data 
    if item < 5:
        group_count.append("count < 5")
    elif item < 10:
        group_count.append("count < 10")
    elif item < 50:
        group_count.append("count < 50")
    else: 
        group_count.append("count >= 50")

    
merge_df["group count"] = group_count
merge_df['group count'].value_counts()

count < 5      667
count < 50     143
count < 10     135
count >= 50    105
Name: group count, dtype: int64

In [7]:
# create the color function 
def select_marker_color(row):
    if row['group count'] == "count < 5":
        return'lightblue'
    elif row['group count'] == "count < 10":
        return 'blue'
    elif row['group count'] == "count < 50":
        return 'darkblue'
    return 'black'
merge_df['color'] = merge_df.apply(select_marker_color,axis =1)


In [8]:
# defining a start point view of the map
my_map = folium.Map(
    location = [32.7767, -96.7970], # dallas
    zoom_start = 6
)

for i in range(len(merge_df['latitude'])):
    folium.Marker(
        location = [merge_df['latitude'][i],merge_df['longitude'][i]],
        popup = merge_df['location'][i],
        tooltip = merge_df['location'][i],
        icon = folium.Icon(color = merge_df['color'][i])
    ).add_to(my_map)
my_map

# my_map.save('data_science_job.html')


In [189]:
# my_map.save('data_science_job.html')

### Q5: What areas have the highest salaries for data scientists?


In [9]:
# merge the data 
merge_df = pd.merge(marker_df,average_comp, on = "location")
merge_df = merge_df.rename(columns = {'totalyearlycompensation':'average yearly compensation'})  
merge_df['average yearly compensation'].describe().to_frame()

,average yearly compensation
count,1050.000000
mean,131689.184849
std,71177.476019
min,10000.000000
25%,87000.000000
50%,121000.000000
75%,158250.000000
max,650000.000000


In [10]:
# defining a start point view of the map
my_map2 = folium.Map(
    zoom_start = 3,
    location = [32.7767, -96.7970], # dallas
    width="%100",
    height="%100"
)

colormap = cm.StepColormap(colors = ['green','yellow','orange','red'],
                           index = [10000,87000,121000,158250,650000],
                           vmin = 10000,
                           vmax = 650000)

# create a Choropleth with the help of folium
for loc, p in zip(zip(merge_df["latitude"],merge_df["longitude"]),merge_df["average yearly compensation"]):
        folium.Circle(
        location=loc,
        radius=2, 
        fill=True, 
        color=colormap(p)).add_to(my_map2)
my_map2


In [11]:
# defining a start point view of the map
my_map3 = folium.Map(
    zoom_start = 4,
    location = [32.7767, -96.7970], # dallas
    width="%100",
    height="%100",
    tiles='stamentoner'
)


# heat map
heat_data = merge_df.groupby(["latitude","longitude"])['average yearly compensation'].mean().reset_index().values.tolist()  
folium.plugins.HeatMap(heat_data).add_to(my_map3)
my_map3
#my_map3.save("heat_map.html")

In [27]:
# color scale
linear = cmp.LinearColormap(
    ['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
    vmin=10000, vmax=650000,
    caption='Color Scale for Map' #Caption for Color scale or Legend
)
linear

### Q4: What areas (states) have the most demand for data scientists?


In [60]:
new_Fyi_Salary['City'] = new_Fyi_Salary['location'].str.split(', ').apply(lambda x: x[0])
new_Fyi_Salary['State'] = new_Fyi_Salary['location'].str.split(', ').apply(lambda x: x[-1])
new_Fyi_Salary.head(5)

,company,title,totalyearlycompensation,location,yearsatcompany,basesalary,City,State
0,Oracle,Product Manager,127000,"Redwood City, CA",1.5,107000.0,Redwood City,CA
1,eBay,Software Engineer,100000,"San Francisco, CA",3.0,0.0,San Francisco,CA
2,Amazon,Product Manager,310000,"Seattle, WA",0.0,155000.0,Seattle,WA
3,Apple,Software Engineering Manager,372000,"Sunnyvale, CA",5.0,157000.0,Sunnyvale,CA
4,Microsoft,Software Engineer,157000,"Mountain View, CA",3.0,0.0,Mountain View,CA


In [20]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

filter_index = []
for i in new_Fyi_Salary["State"]:
    filter_index.append(i in states)
        
usa_df = new_Fyi_Salary.loc[filter_index,:]
usa_df.head()


,company,title,totalyearlycompensation,location,yearsatcompany,basesalary,City,State
0,Oracle,Product Manager,127000,"Redwood City, CA",1.5,107000.0,Redwood City,CA
1,eBay,Software Engineer,100000,"San Francisco, CA",3.0,0.0,San Francisco,CA
2,Amazon,Product Manager,310000,"Seattle, WA",0.0,155000.0,Seattle,WA
3,Apple,Software Engineering Manager,372000,"Sunnyvale, CA",5.0,157000.0,Sunnyvale,CA
4,Microsoft,Software Engineer,157000,"Mountain View, CA",3.0,0.0,Mountain View,CA


In [52]:
usa_df_group = usa_df.groupby(('State'), as_index=False)
count_usa = usa_df_group["company"].count()
count_usa = count_usa.rename(columns = {'company':'count'})  
count_usa

,State,count
0,AL,25
1,AR,88
2,AZ,379
3,CA,22824
4,CO,590
5,CT,106
6,DC,592
7,DE,59
8,FL,298
9,GA,572


In [57]:
fig = go.Figure(data=go.Choropleth(
    locations=count_usa['State'], # Spatial coordinates
    z = count_usa['count'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Greens',
    colorbar_title = "Total Workers",
))

fig.update_layout(
    title_text = '',
    geo_scope='usa', # limite map scope to USA
)

fig.show()